# Visualize an M/G/1 queue

We first replace the `arrival_rate` and `service_rate` parameters with `arrival_distribution` and `service_distribution` parameters so we can generate samples from them during the simulation. This makes the simulation more general since we can pass in _any_ distribution as a parameter.

## Event and Schedule classes: Unchanged

In [1]:
import heapq

class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run
    
    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

## Queue and BusSystem classes

The changes are all related to the `service_distribution` and `arrival_distribution` variables below.

In [2]:
# M/D/1 queue

import scipy.stats as sts


class Queue:
    
    def __init__(self, service_distribution):
        # Store the deterministic service time for an M/D/1 queue
        self.service_distribution = service_distribution
        # We start with an empty queue and the server not busy
        self.people_in_queue = 0
        self.people_being_served = 0

    def add_customer(self, schedule):
        # Add the customer to the queue
        self.people_in_queue += 1
        print(
            f'{schedule.now:5.2f}: Add customer to queue.  '
            f'Queue length: {self.people_in_queue}')
        if self.people_being_served < 1:
            # This customer can be served immediately
            schedule.add_event_after(0, self.start_serving_customer)
            
    def start_serving_customer(self, schedule):
        # Move the customer from the queue to a server
        self.people_in_queue -= 1
        self.people_being_served += 1
        print(
            f'{schedule.now:5.2f}: Start serving customer. '
            f'Queue length: {self.people_in_queue}')
        # Schedule when the server will be done with the customer.
        # Generate a random service time from the service distribution.
        schedule.add_event_after(
            self.service_distribution.rvs(),
            self.finish_serving_customer)
            
    def finish_serving_customer(self, schedule):
        # Remove the customer from the server
        self.people_being_served -= 1
        print(
            f'{schedule.now:5.2f}: Stop serving customer.  '
            f'Queue length: {self.people_in_queue}')
        if self.people_in_queue > 0:
            # There are more people in the queue so serve the next customer
            schedule.add_event_after(0, self.start_serving_customer)


class BusSystem:
    
    def __init__(self, arrival_distribution, service_distribution):
        self.queue = Queue(service_distribution)
        self.arrival_distribution = arrival_distribution

    def add_customer(self, schedule):
        # Add this customer to the queue
        self.queue.add_customer(schedule)
        # Schedule when to add another customer
        schedule.add_event_after(
            self.arrival_distribution.rvs(),
            self.add_customer)

    def run(self, schedule):
        # Schedule when the first customer arrives
        schedule.add_event_after(
            self.arrival_distribution.rvs(),
            self.add_customer)
        

def run_simulation(arrival_distribution, service_distribution, run_until):
    schedule = Schedule()
    bus_system = BusSystem(arrival_distribution, service_distribution)
    bus_system.run(schedule)
    while schedule.next_event_time() < run_until:
        schedule.run_next_event()
    return bus_system

## Run an M/D/1 simulation

This is the original distribution with a deterministic service time.

In [3]:
arrival_rate = 1.2
arrival_distribution = sts.expon(scale=1/arrival_rate)

# This is a trick for getting a deterministic distribution. We
# set the standard deviation of a normal distribution to 0, which
# means all the probability mass is concentrated on the mean and
# every random sample from the distribution will be equal to the mean.
service_rate = 1
service_distribution = sts.norm(loc=1/service_rate, scale=0)

# Run the simulation once
duration = 100
bus_system = run_simulation(arrival_distribution, service_distribution, duration)
print(f'There are {bus_system.queue.people_in_queue} people in the queue')

 1.33: Add customer to queue.  Queue length: 1
 1.33: Start serving customer. Queue length: 0
 1.65: Add customer to queue.  Queue length: 1
 1.78: Add customer to queue.  Queue length: 2
 1.91: Add customer to queue.  Queue length: 3
 2.16: Add customer to queue.  Queue length: 4
 2.33: Stop serving customer.  Queue length: 4
 2.33: Start serving customer. Queue length: 3
 2.49: Add customer to queue.  Queue length: 4
 3.33: Stop serving customer.  Queue length: 4
 3.33: Start serving customer. Queue length: 3
 4.33: Stop serving customer.  Queue length: 3
 4.33: Start serving customer. Queue length: 2
 4.62: Add customer to queue.  Queue length: 3
 5.33: Stop serving customer.  Queue length: 3
 5.33: Start serving customer. Queue length: 2
 6.17: Add customer to queue.  Queue length: 3
 6.33: Stop serving customer.  Queue length: 3
 6.33: Start serving customer. Queue length: 2
 6.57: Add customer to queue.  Queue length: 3
 7.33: Stop serving customer.  Queue length: 3
 7.33: Start 

## Run an M/G/1 simulation

We replace the distribution distribution with a more general normal distribution.

In [4]:
# Exponential with lambda = 1.2
arrival_distribution = sts.expon(scale=1/1.2)

# Normal with mu = 3 and sigma = 1
service_distribution = sts.norm(loc=3, scale=1)

duration = 100
bus_system = run_simulation(arrival_distribution, service_distribution, duration)
print(f'There are {bus_system.queue.people_in_queue} people in the queue')

 1.01: Add customer to queue.  Queue length: 1
 1.01: Start serving customer. Queue length: 0
 3.02: Add customer to queue.  Queue length: 1
 4.06: Stop serving customer.  Queue length: 1
 4.06: Start serving customer. Queue length: 0
 5.30: Stop serving customer.  Queue length: 0
 5.74: Add customer to queue.  Queue length: 1
 5.74: Start serving customer. Queue length: 0
 6.67: Add customer to queue.  Queue length: 1
 8.21: Add customer to queue.  Queue length: 2
 9.01: Add customer to queue.  Queue length: 3
10.43: Stop serving customer.  Queue length: 3
10.43: Start serving customer. Queue length: 2
11.61: Add customer to queue.  Queue length: 3
12.40: Add customer to queue.  Queue length: 4
12.54: Add customer to queue.  Queue length: 5
13.02: Add customer to queue.  Queue length: 6
13.22: Add customer to queue.  Queue length: 7
13.33: Add customer to queue.  Queue length: 8
14.39: Add customer to queue.  Queue length: 9
15.07: Stop serving customer.  Queue length: 9
15.07: Start 